## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [6]:
import numpy as np
import pandas as pd
import random
import string
a_array = np.random.randint(0,1001, size = 1000000)
b_array = np.array([i+100 for i in a_array])
print(a_array)
print(b_array)
print('Среднее значение =',np.mean(b_array))

[943 643 850 ... 165 427 415]
[1043  743  950 ...  265  527  515]
Среднее значение = 599.74335


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [37]:
key = [''.join(random.choices(string.ascii_letters, k=5)).lower() for _ in range(2000000)]
df = pd.DataFrame(np.random.randint(0,1001, size=(2000000,4)))
df['key'] = key
df

,0,1,2,3,key
0,920,266,109,921,imbre
1,919,60,535,399,zazcy
2,270,189,258,785,mmmvi
3,320,477,268,372,gueko
4,331,733,891,803,wsczm
...,...,...,...,...,...
1999995,161,805,395,724,jtkzz
1999996,602,237,755,529,lkzrz
1999997,970,980,591,962,lhqnj
1999998,505,36,953,58,ymqek


In [38]:
if df.loc[df['key'] == 'abcde'].shape[0] == 0:
    print(df.loc[df['key']=='abcde'])
    print('Нет строк с ключем, содержащим первые 5 букв английского алфавита')
else:print(df.loc[df['key']=='abcde'])

Empty DataFrame
Columns: [0, 1, 2, 3, key]
Index: []
Нет строк с ключем, содержащим первые 5 букв английского алфавита


## Лабораторная работа 3

In [39]:
!pip install line_profiler

     -------------------------------------- 83.6/83.6 kB 940.1 kB/s eta 0:00:00


In [40]:
%load_ext line_profiler

In [61]:
import timeit

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [76]:
recipes = pd.read_csv('recipes_sample.csv',delimiter=',')
# recipes
reviews = pd.read_csv('reviews_sample.csv',delimiter=',')
reviews.rename(columns = {'Unnamed: 0':'id'}, inplace = True)
df=pd.merge(recipes, reviews)
df['submitted'], df['date'] = pd.to_datetime(df['submitted']),pd.to_datetime(df['date'])
df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,user_id,recipe_id,date,rating,review
0,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2000576480,445211,2015-10-18,1,Terrible - makes a very runny batter like for ...
1,say what banana sandwich,95926,5,118163,2004-07-20,4.0,you just have to try it to believe it.,NaN,221694,126623,2009-02-01,4,This was yummy. I cut the entire sauce recipe ...
2,add in anything muffins,149593,15,89831,2005-12-28,NaN,"this is a never-fail muffin recipe, it's a bla...",9.0,52282,66815,2008-07-12,5,"my family all enjoyed this dish, and yes it do..."
3,burek or feta cheese phyllo pie,310570,65,676820,2008-06-24,38.0,"ok, there are different version of burek (some...",6.0,527607,260529,2007-10-23,5,These were fantastic. I lightened it even fur...
4,skordy new potatoes w rosemary lemon olive oi,296983,35,718054,2008-04-08,NaN,i took this recipe from a vegan tastes of gree...,6.0,795795,50385,2008-03-20,5,This recipe was WONDERFULLY DELICIOUS! The rev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,zucchini meat sauce with pasta,253264,30,275742,2007-09-17,NaN,havn't tried this recipe yet but hopefully wil...,6.0,142849,102617,2007-08-01,5,This is fabulous! I do cheat however and use t...
3447,zucchini stuffed with feta and basil,72857,30,95743,2003-10-08,NaN,"easy to make, savory and a nice presentation. ...",5.0,49937,54269,2003-04-20,5,This cake mix extender is great! I used it wit...
3448,zucchini tomato salsa,321224,20,464080,2008-08-25,4.0,a contest winner from taste of home. it state...,13.0,171790,394467,2010-01-11,5,I loved this soup! The only changes I made wer...
3449,zucchini potato and herb fritters,344542,30,197023,2008-12-21,8.0,adapted from a recipe in 'the australian women...,12.0,659599,222188,2013-10-15,5,So simple but yet so delicious. I only made h...


In [95]:
#A
def find_average():
    lst_rating = [row.rating for index,row in df.iterrows() if row.submitted.year == 2010]
    average_rating = sum(lst_rating) / df.loc[df['submitted'].dt.year == 2010].shape[0] #деление на общее число отзывов, оставленных к 2010
    return average_rating
%prun av=find_average() 
print(av)
execution_time=timeit.timeit(find_average, number=1)
print(f'Время исполнения - {execution_time}')

 4.270440251572327
Время исполнения - 0.15714699999989534


In [96]:
#B
def find_average_b():
    lst_rating = [row.rating for index,row in df.loc[df['submitted'].dt.year == 2010].iterrows()]
    average_rating = sum(lst_rating) / df.loc[df['submitted'].dt.year == 2010].shape[0]
    return average_rating
%prun av1=find_average_b()
print(av1)
execution_time=timeit.timeit(find_average_b, number=1)
print(f'Время исполнения - {execution_time}')

 4.270440251572327
Время исполнения - 0.009748399999807589


In [97]:
#C
def find_average_c():
    return df.loc[df['submitted'].dt.year == 2010].rating.mean()
%prun av2=find_average_c()
print(av2)
execution_time=timeit.timeit(find_average_c, number=1)
print(f'Время исполнения - {execution_time}')

 4.270440251572327
Время исполнения - 0.002245499999844469


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

**вариант A**:

489983 function calls (483060 primitive calls) in 0.853 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.504    0.168    0.504    0.168 {method 'astype' of 'numpy.ndarray' objects}
     3453    0.024    0.000    0.185    0.000 series.py:323(__init__)
        1    0.021    0.021    0.021    0.021 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
   111663    0.014    0.000    0.018    0.000 {built-in method builtins.isinstance}
     3453    0.013    0.000    0.054    0.000 construction.py:470(sanitize_array)
     3451    0.009    0.000    0.016    0.000 cast.py:1466(maybe_infer_to_datetimelike)
        5    0.009    0.002    0.009    0.002 {built-in method numpy.empty}
     3453    0.008    0.000    0.026    0.000 blocks.py:2043(new_block)
     3610    0.008    0.000    0.040    0.000 series.py:943(__getitem__)
     7065    0.008    0.000    0.067    0.000 generic.py:5561(__getattr__)
     3458    0.007    0.000    0.025    0.000 generic.py:5577(__setattr__)
     3454    0.007    0.000    0.009    0.000 generic.py:239(__init__)
41486/34571    0.007    0.000    0.010    0.000 {built-in method builtins.len}
     3452    0.007    0.000    0.753    0.000 frame.py:1279(iterrows)
        1    0.007    0.007    0.825    0.825 2675988602.py:3(<listcomp>)
     3453    0.006    0.000    0.013    0.000 config.py:109(_get_single_key)
     3453    0.005    0.000    0.022    0.000 construction.py:698(_try_cast)
     3453    0.005    0.000    0.035    0.000 managers.py:1745(from_array)
     3451    0.005    0.000    0.005    0.000 {pandas._libs.lib.infer_datetimelike_array}
        1    0.004    0.004    0.558    0.558 managers.py:1611(_interleave)
     3610    0.004    0.000    0.025    0.000 series.py:1052(_get_value)
     3611    0.004    0.000    0.005    0.000 base.py:4987(__contains__)
     3456    0.004    0.000    0.012    0.000 series.py:640(name)
     3453    0.004    0.000    0.005    0.000 blocks.py:1991(get_block_type)
     3453    0.004    0.000    0.007    0.000 blocks.py:1962(maybe_coerce_values)
     3610    0.004    0.000    0.011    0.000 base.py:3585(get_loc)
     3613    0.004    0.000    0.004    0.000 managers.py:1862(internal_values)
     3457    0.003    0.000    0.006    0.000 series.py:590(name)
     3453    0.003    0.000    0.004    0.000 config.py:589(_get_root)
     3453    0.003    0.000    0.005    0.000 blocks.py:2057(check_ndim)
     3610    0.003    0.000    0.010    0.000 base.py:5660(_get_values_for_loc)
     3615    0.003    0.000    0.008    0.000 common.py:161(is_object_dtype)
     3453    0.003    0.000    0.020    0.000 config.py:127(_get_option)
     6906    0.003    0.000    0.003    0.000 config.py:603(_get_deprecated_option)
    10400    0.003    0.000    0.004    0.000 generic.py:43(_check)
17482/17477    0.003    0.000    0.007    0.000 {built-in method builtins.getattr}
     6915    0.003    0.000    0.004    0.000 base.py:884(__len__)
     3610    0.003    0.000    0.017    0.000 base.py:5092(_can_hold_identifiers_and_holds_name)
     3453    0.003    0.000    0.005    0.000 construction.py:630(_sanitize_ndim)
     3610    0.003    0.000    0.006    0.000 base.py:6298(_maybe_cast_indexer)
     6913    0.002    0.000    0.003    0.000 inference.py:321(is_hashable)
     3453    0.002    0.000    0.004    0.000 construction.py:805(is_empty_data)
     3621    0.002    0.000    0.005    0.000 common.py:1587(_is_dtype_type)
     3612    0.002    0.000    0.004    0.000 indexing.py:2486(check_deprecated_indexers)
     3456    0.002    0.000    0.008    0.000 common.py:1721(validate_all_hashable)
     3610    0.002    0.000    0.002    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
     3454    0.002    0.000    0.002    0.000 flags.py:47(__init__)
        1    0.002    0.002    0.853    0.853 2675988602.py:2(find_average)
     3458    0.002    0.000    0.006    0.000 {built-in method builtins.all}
     3453    0.002    0.000    0.022    0.000 config.py:255(__call__)
       20    0.002    0.000    0.002    0.000 datetimes.py:545(dtype)
        1    0.002    0.002    0.009    0.009 cast.py:1789(find_common_type)
     6912    0.002    0.000    0.004    0.000 common.py:1740(<genexpr>)
     3610    0.002    0.000    0.010    0.000 base.py:2440(is_object)
     3613    0.002    0.000    0.006    0.000 series.py:687(_values)
        1    0.002    0.002    0.002    0.002 {pandas._libs.lib.dtypes_all_equal}
     3453    0.002    0.000    0.002    0.000 managers.py:1714(__init__)
     3455    0.002    0.000    0.009    0.000 construction.py:379(extract_array)
     3456    0.002    0.000    0.002    0.000 construction.py:438(ensure_wrapped_if_datetimelike)
    10365    0.002    0.000    0.002    0.000 {pandas._libs.lib.is_list_like}
    10524    0.002    0.000    0.002    0.000 {built-in method builtins.hash}
        2    0.002    0.001    0.002    0.001 numerictypes.py:575(_can_coerce_all)
        1    0.002    0.002    0.005    0.005 series.py:5613(_cmp_method)
     3453    0.001    0.000    0.004    0.000 common.py:552(require_length_match)
     3455    0.001    0.000    0.003    0.000 base.py:7099(maybe_extract_name)
        1    0.001    0.001    0.002    0.002 datetimes.py:2019(_sequence_to_dt64ns)
     3453    0.001    0.000    0.003    0.000 config.py:642(_warn_if_deprecated)
    10539    0.001    0.000    0.001    0.000 {built-in method builtins.issubclass}
        3    0.001    0.000    0.002    0.001 base.py:654(_simple_new)
        7    0.001    0.000    0.002    0.000 common.py:1274(is_bool_dtype)
     3610    0.001    0.000    0.001    0.000 base.py:2352(is_floating)
     3453    0.001    0.000    0.001    0.000 series.py:542(_set_axis)
     3453    0.001    0.000    0.002    0.000 construction.py:687(_maybe_repeat)
     3610    0.001    0.000    0.002    0.000 common.py:160(cast_scalar_indexer)
     3453    0.001    0.000    0.002    0.000 construction.py:667(_sanitize_str_dtypes)
     3610    0.001    0.000    0.002    0.000 generic.py:636(_info_axis)
        1    0.001    0.001    0.561    0.561 frame.py:10817(values)
     3453    0.001    0.000    0.002    0.000 config.py:630(_translate_key)
        1    0.001    0.001    0.001    0.001 {pandas._libs.tslibs.fields.get_date_field}
        1    0.001    0.001    0.029    0.029 blocks.py:1463(get_values)
     3452    0.001    0.000    0.001    0.000 {built-in method numpy.array}
        1    0.001    0.001    0.001    0.001 datetimes.py:496(_unbox_scalar)
     3612    0.001    0.000    0.002    0.000 common.py:346(apply_if_callable)
        2    0.001    0.001    0.001    0.001 {pandas._libs.algos.take_2d_axis1_int64_int64}
     3453    0.001    0.000    0.001    0.000 base.py:7004(ensure_index)
        2    0.001    0.000    0.001    0.001 dtypes.py:955(is_dtype)
        7    0.001    0.000    0.001    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     7220    0.001    0.000    0.001    0.000 {pandas._libs.lib.is_integer}
     3453    0.001    0.000    0.001    0.000 {method 'split' of 'str' objects}
     3617    0.001    0.000    0.001    0.000 common.py:147(<lambda>)
        1    0.001    0.001    0.002    0.002 datetimelike.py:1911(maybe_infer_freq)
     3453    0.001    0.000    0.001    0.000 config.py:571(_select_options)
        1    0.001    0.001    0.001    0.001 datetimelike.py:836(_isnan)
     3453    0.001    0.000    0.001    0.000 common.py:1416(is_1d_only_ea_dtype)
     3451    0.001    0.000    0.001    0.000 {pandas._libs.algos.ensure_object}
        4    0.001    0.000    0.001    0.000 blocks.py:244(mgr_locs)
        5    0.001    0.000    0.001    0.000 {built-in method pandas._libs.missing.checknull}
     3617    0.001    0.000    0.001    0.000 common.py:145(classes)
     3453    0.001    0.000    0.001    0.000 managers.py:1806(_block)
        1    0.001    0.001    0.001    0.001 {built-in method pandas._libs.tslibs.offsets.to_offset}
     3626    0.001    0.000    0.001    0.000 {pandas._libs.lib.is_scalar}
     6907    0.001    0.000    0.001    0.000 typing.py:1375(cast)
        1    0.001    0.001    0.001    0.001 utils.py:249(maybe_convert_indices)
        1    0.001    0.001    0.001    0.001 indexing.py:267(loc)
        1    0.001    0.001    0.001    0.001 {built-in method _operator.eq}
        1    0.001    0.001    0.023    0.023 datetimelike.py:403(astype)
     3468    0.001    0.000    0.001    0.000 {built-in method builtins.hasattr}
        1    0.001    0.001    0.001    0.001 numerictypes.py:651(<listcomp>)
        3    0.001    0.000    0.001    0.000 common.py:1747(pandas_dtype)
        1    0.001    0.001    0.003    0.003 array_ops.py:227(comparison_op)
        3    0.001    0.000    0.001    0.000 missing.py:572(na_value_for_dtype)
    17/15    0.001    0.000    0.001    0.000 {built-in method numpy.asarray}
     3612    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
        1    0.000    0.000    0.000    0.000 managers.py:1625(<listcomp>)
        1    0.000    0.000    0.001    0.001 array_ops.py:134(_na_arithmetic_op)
        1    0.000    0.000    0.027    0.027 datetimes.py:645(astype)
     3610    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_float}
        3    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x00007FFE9B8EDC60}
        2    0.000    0.000    0.002    0.001 algorithms.py:1356(take)
        1    0.000    0.000    0.013    0.013 accessor.py:79(_getter)
        1    0.000    0.000    0.013    0.013 accessors.py:79(_delegate_property_get)
        4    0.000    0.000    0.000    0.000 base.py:286(is_dtype)
        1    0.000    0.000    0.000    0.000 base.py:744(__iter__)
        2    0.000    0.000    0.001    0.000 generic.py:5650(f)
        2    0.000    0.000    0.001    0.001 generic.py:5632(_protect_consolidate)
        1    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis1_float64_float64}
        1    0.000    0.000    0.003    0.003 datetimes.py:115(f)
        1    0.000    0.000    0.007    0.007 accessors.py:61(_get_values)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000    0.000    0.006    0.006 datetimes.py:305(__new__)
        1    0.000    0.000    0.559    0.559 managers.py:1551(as_array)
        7    0.000    0.000    0.000    0.000 common.py:581(is_dtype_equal)
        1    0.000    0.000    0.003    0.003 numerictypes.py:599(find_common_type)
        1    0.000    0.000    0.000    0.000 {built-in method _simple_new}
        3    0.000    0.000    0.001    0.000 datetimelike.py:280(asi8)
        3    0.000    0.000    0.001    0.000 cast.py:515(_maybe_promote)
        9    0.000    0.000    0.000    0.000 managers.py:1851(dtype)
        1    0.000    0.000    0.000    0.000 datetimes.py:96(tz_to_dtype)
        5    0.000    0.000    0.000    0.000 {method 'view' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.009    0.009 base.py:210(interleaved_dtype)
        5    0.000    0.000    0.000    0.000 common.py:315(is_datetime64_dtype)
        1    0.000    0.000    0.000    0.000 base.py:1106(take)
        1    0.000    0.000    0.001    0.001 utils.py:197(validate_indices)
      5/4    0.000    0.000    0.005    0.001 take.py:57(take_nd)
        5    0.000    0.000    0.001    0.000 missing.py:150(_isna)
        1    0.000    0.000    0.000    0.000 common.py:925(is_datetime64_ns_dtype)
       19    0.000    0.000    0.000    0.000 numerictypes.py:584(<listcomp>)
        1    0.000    0.000    0.004    0.004 datetimes.py:341(_from_sequence_not_strict)
        1    0.000    0.000    0.000    0.000 datetimes.py:2268(maybe_convert_dtype)
        1    0.000    0.000    0.000    0.000 base.py:554(_dtype_to_subclass)
        1    0.000    0.000    0.005    0.005 arraylike.py:38(__eq__)
        1    0.000    0.000    0.000    0.000 {built-in method fromkeys}
        1    0.000    0.000    0.001    0.001 base.py:397(__new__)
        1    0.000    0.000    0.000    0.000 indexing.py:2357(check_bool_indexer)
        1    0.000    0.000    0.001    0.001 expressions.py:92(_evaluate_numexpr)
        1    0.000    0.000    0.000    0.000 numeric.py:2385(array_equal)
        1    0.000    0.000    0.001    0.001 numeric.py:319(_assert_safe_casting)
        4    0.000    0.000    0.000    0.000 missing.py:626(is_valid_na_for_dtype)
        1    0.000    0.000    0.000    0.000 utils.py:457(check_array_indexer)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:32(seterr)
        1    0.000    0.000    0.000    0.000 numeric.py:2381(_array_equal_dispatcher)
        1    0.000    0.000    0.006    0.006 generic.py:3609(take)
        4    0.000    0.000    0.001    0.000 {method 'any' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:191(is_sparse)
        2    0.000    0.000    0.001    0.001 generic.py:5646(_consolidate_inplace)
        3    0.000    0.000    0.000    0.000 generic.py:5517(__finalize__)
        1    0.000    0.000    0.000    0.000 frame.py:3463(__getitem__)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(array_equal)
        1    0.000    0.000    0.005    0.005 common.py:55(new_method)
        2    0.000    0.000    0.000    0.000 managers.py:618(consolidate)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:131(geterr)
        4    0.000    0.000    0.005    0.001 blocks.py:1116(take_nd)
        4    0.000    0.000    0.000    0.000 take.py:326(_get_take_nd_function)
       23    0.000    0.000    0.000    0.000 common.py:1552(get_dtype)
        4    0.000    0.000    0.002    0.000 take.py:120(_take_nd_ndarray)
        1    0.000    0.000    0.007    0.007 indexing.py:954(__getitem__)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:425(__init__)
        2    0.000    0.000    0.001    0.001 common.py:423(is_period_dtype)
        1    0.000    0.000    0.004    0.004 extension.py:67(fget)
        3    0.000    0.000    0.000    0.000 cast.py:1828(<genexpr>)
        1    0.000    0.000    0.001    0.001 numeric.py:139(_ensure_array)
        1    0.000    0.000    0.006    0.006 managers.py:872(take)
        1    0.000    0.000    0.003    0.003 _mixins.py:147(take)
        1    0.000    0.000    0.000    0.000 series.py:2988(_construct_result)
        1    0.000    0.000    0.001    0.001 expressions.py:223(evaluate)
        3    0.000    0.000    0.504    0.168 blocks.py:222(get_values)
        1    0.000    0.000    0.000    0.000 frame.py:3923(_get_item_cache)
        1    0.000    0.000    0.853    0.853 {built-in method builtins.exec}
        4    0.000    0.000    0.001    0.000 _methods.py:54(_any)
        3    0.000    0.000    0.002    0.001 base.py:53(shape)
        4    0.000    0.000    0.000    0.000 blocks.py:267(make_block_same_class)
        1    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis1_object_object}
        1    0.000    0.000    0.007    0.007 indexing.py:1169(_getitem_axis)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:429(__enter__)
        6    0.000    0.000    0.001    0.000 cast.py:468(maybe_promote)
        5    0.000    0.000    0.001    0.000 missing.py:67(isna)
        1    0.000    0.000    0.000    0.000 _ufunc_config.py:434(__exit__)
        1    0.000    0.000    0.000    0.000 datetimes.py:326(_simple_new)
        1    0.000    0.000    0.000    0.000 dispatch.py:11(should_extension_dispatch)
        1    0.000    0.000    0.001    0.001 take.py:350(wrapper)
        9    0.000    0.000    0.000    0.000 series.py:575(dtype)
        3    0.000    0.000    0.000    0.000 datetimes.py:570(tz)
        1    0.000    0.000    0.000    0.000 base.py:465(find)
        3    0.000    0.000    0.000    0.000 flags.py:83(allows_duplicate_labels)
        1    0.000    0.000    0.002    0.002 datetimelike.py:850(_maybe_mask_results)
        1    0.000    0.000    0.000    0.000 frame.py:587(__init__)
        1    0.000    0.000    0.007    0.007 indexing.py:981(_getbool_axis)
        1    0.000    0.000    0.002    0.002 datetimelike.py:843(_hasna)
        2    0.000    0.000    0.000    0.000 base.py:5192(equals)
        4    0.000    0.000    0.000    0.000 generic.py:560(_get_axis)
        2    0.000    0.000    0.000    0.000 managers.py:1675(is_consolidated)
        1    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 managers.py:970(from_blocks)
        1    0.000    0.000    0.000    0.000 common.py:105(is_bool_indexer)
        1    0.000    0.000    0.001    0.001 datetimelike.py:591(_validate_scalar)
        2    0.000    0.000    0.000    0.000 common.py:1240(is_float_dtype)
        1    0.000    0.000    0.000    0.000 missing.py:138(dispatch_fill_zeros)
        1    0.000    0.000    0.001    0.001 expressions.py:63(_evaluate_standard)
        3    0.000    0.000    0.000    0.000 common.py:1429(is_extension_array_dtype)
        4    0.000    0.000    0.000    0.000 inference.py:184(is_array_like)
        1    0.000    0.000    0.000    0.000 common.py:1042(is_numeric_v_string_like)
        4    0.000    0.000    0.000    0.000 take.py:554(_take_preprocess_indexer_and_fill_value)
        1    0.000    0.000    0.000    0.000 {method 'all' of 'numpy.ndarray' objects}
        5    0.000    0.000    0.000    0.000 cast.py:1819(<genexpr>)
        1    0.000    0.000    0.005    0.005 managers.py:634(reindex_indexer)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.005    0.005 managers.py:689(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        3    0.000    0.000    0.000    0.000 generic.py:328(attrs)
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 expressions.py:72(_can_use_numexpr)
        4    0.000    0.000    0.000    0.000 {built-in method numpy.geterrobj}
        1    0.000    0.000    0.000    0.000 managers.py:217(is_single_block)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.seterrobj}
        7    0.000    0.000    0.000    0.000 base.py:4834(_values)
        2    0.000    0.000    0.000    0.000 cast.py:1830(<genexpr>)
        3    0.000    0.000    0.000    0.000 flags.py:51(allows_duplicate_labels)
        1    0.000    0.000    0.000    0.000 common.py:680(is_integer_dtype)
        1    0.000    0.000    0.000    0.000 function.py:49(__call__)
        1    0.000    0.000    0.006    0.006 generic.py:3708(_take_with_is_copy)
        1    0.000    0.000    0.000    0.000 frame.py:578(_constructor)
        1    0.000    0.000    0.000    0.000 datetimes.py:505(_check_compatible_with)
        9    0.000    0.000    0.002    0.000 base.py:55(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'take' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'transpose' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:75(get_op_result_name)
        1    0.000    0.000    0.000    0.000 datetimes.py:795(_local_timestamps)
        5    0.000    0.000    0.002    0.000 cast.py:1835(<genexpr>)
        2    0.000    0.000    0.002    0.001 {built-in method builtins.any}
        1    0.000    0.000    0.853    0.853 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 common.py:1399(is_1d_only_ea_obj)
        5    0.000    0.000    0.000    0.000 generic.py:546(_get_axis_number)
        3    0.000    0.000    0.000    0.000 base.py:834(_reset_identity)
        2    0.000    0.000    0.000    0.000 cast.py:1711(sanitize_to_nanoseconds)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        2    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {built-in method pandas._libs.tslibs.timezones.maybe_get_tz}
        4    0.000    0.000    0.000    0.000 common.py:155(<lambda>)
        2    0.000    0.000    0.000    0.000 common.py:1483(is_ea_or_datetimelike_dtype)
        1    0.000    0.000    0.000    0.000 {method 'min' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 numeric.py:188(_validate_dtype)
        4    0.000    0.000    0.000    0.000 {pandas._libs.lib.item_from_zerodim}
        2    0.000    0.000    0.000    0.000 common.py:389(is_timedelta64_dtype)
        1    0.000    0.000    0.000    0.000 common.py:540(is_string_dtype)
        2    0.000    0.000    0.000    0.000 datetimelike.py:880(freq)
        1    0.000    0.000    0.000    0.000 generic.py:3925(_set_is_copy)
        2    0.000    0.000    0.000    0.000 common.py:732(is_signed_integer_dtype)
        1    0.000    0.000    0.000    0.000 common.py:348(is_datetime64tz_dtype)
        3    0.000    0.000    0.000    0.000 cast.py:454(ensure_dtype_can_hold_na)
        1    0.000    0.000    0.000    0.000 datetimes.py:2359(_validate_dt64_dtype)
        1    0.000    0.000    0.000    0.000 base.py:1132(_maybe_disallow_fill)
        8    0.000    0.000    0.000    0.000 base.py:326(ndim)
        1    0.000    0.000    0.000    0.000 blocks.py:350(shape)
        3    0.000    0.000    0.000    0.000 common.py:1148(needs_i8_conversion)
        5    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_platform_int}
        1    0.000    0.000    0.000    0.000 {method 'max' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 _methods.py:42(_amin)
        2    0.000    0.000    0.000    0.000 base.py:803(is_)
        1    0.000    0.000    0.000    0.000 series.py:825(__array__)
        1    0.000    0.000    0.000    0.000 {method '_from_backing_data' of 'pandas._libs.arrays.NDArrayBacked' objects}
        1    0.000    0.000    0.000    0.000 common.py:786(is_unsigned_integer_dtype)
        1    0.000    0.000    0.000    0.000 missing.py:391(array_equivalent)
        1    0.000    0.000    0.000    0.000 frame.py:821(shape)
        6    0.000    0.000    0.000    0.000 generic.py:349(flags)
        1    0.000    0.000    0.000    0.000 blocks.py:354(dtype)
        2    0.000    0.000    0.000    0.000 base.py:1658(name)
        1    0.000    0.000    0.000    0.000 common.py:497(is_categorical_dtype)
        1    0.000    0.000    0.000    0.000 base.py:890(__array__)
        4    0.000    0.000    0.000    0.000 common.py:150(classes_and_not_datetimelike)
        1    0.000    0.000    0.000    0.000 common.py:569(condition)
        2    0.000    0.000    0.000    0.000 datetimes.py:2401(validate_tz_from_dtype)
        1    0.000    0.000    0.000    0.000 generic.py:566(_get_block_manager_axis)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asanyarray}
        1    0.000    0.000    0.000    0.000 common.py:1528(_is_dtype)
        1    0.000    0.000    0.000    0.000 datetimelike.py:1876(validate_inferred_freq)
        1    0.000    0.000    0.000    0.000 base.py:945(dtype)
        1    0.000    0.000    0.000    0.000 managers.py:156(blknos)
        1    0.000    0.000    0.000    0.000 numeric.py:199(_ensure_dtype)
        1    0.000    0.000    0.000    0.000 _methods.py:38(_amax)
        1    0.000    0.000    0.000    0.000 _methods.py:60(_all)
        1    0.000    0.000    0.000    0.000 series.py:523(_constructor)
        2    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_iterator}
        1    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 managers.py:172(blklocs)
        1    0.000    0.000    0.000    0.000 numerictypes.py:652(<listcomp>)
        1    0.000    0.000    0.000    0.000 base.py:229(disallow_kwargs)
        2    0.000    0.000    0.000    0.000 numeric.py:331(_is_all_dates)
        1    0.000    0.000    0.000    0.000 managers.py:916(__init__)

**Вариант B**

24568 function calls (24214 primitive calls) in 0.040 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.011    0.005    0.011    0.005 {pandas._libs.algos.take_2d_axis1_object_object}
        3    0.003    0.001    0.003    0.001 {method 'astype' of 'numpy.ndarray' objects}
      163    0.001    0.000    0.012    0.000 series.py:323(__init__)
        9    0.001    0.000    0.001    0.000 {built-in method numpy.empty}
        1    0.001    0.001    0.005    0.005 managers.py:1611(_interleave)
      163    0.001    0.000    0.001    0.000 base.py:7004(ensure_index)
     5594    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
      338    0.001    0.000    0.001    0.000 base.py:884(__len__)
      161    0.001    0.000    0.001    0.000 base.py:4987(__contains__)
      163    0.001    0.000    0.003    0.000 construction.py:470(sanitize_array)
        2    0.001    0.000    0.001    0.000 {pandas._libs.tslibs.fields.get_date_field}
      159    0.001    0.000    0.001    0.000 cast.py:1466(maybe_infer_to_datetimelike)
      163    0.000    0.000    0.001    0.000 blocks.py:2043(new_block)
      159    0.000    0.000    0.002    0.000 series.py:943(__getitem__)
        1    0.000    0.000    0.019    0.019 869757136.py:3(<listcomp>)
       14    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      165    0.000    0.000    0.001    0.000 generic.py:239(__init__)
      326    0.000    0.000    0.004    0.000 generic.py:5561(__getattr__)
      172    0.000    0.000    0.001    0.000 generic.py:5577(__setattr__)
        1    0.000    0.000    0.000    0.000 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
2021/1683    0.000    0.000    0.001    0.000 {built-in method builtins.len}
      163    0.000    0.000    0.001    0.000 construction.py:698(_try_cast)
      165    0.000    0.000    0.000    0.000 managers.py:1862(internal_values)
        4    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis1_int64_int64}
      160    0.000    0.000    0.014    0.000 frame.py:1279(iterrows)
      163    0.000    0.000    0.002    0.000 managers.py:1745(from_array)
      163    0.000    0.000    0.001    0.000 config.py:109(_get_single_key)
      168    0.000    0.000    0.001    0.000 common.py:161(is_object_dtype)
      159    0.000    0.000    0.001    0.000 series.py:1052(_get_value)
      163    0.000    0.000    0.000    0.000 blocks.py:1991(get_block_type)
      159    0.000    0.000    0.000    0.000 {pandas._libs.lib.infer_datetimelike_array}
      169    0.000    0.000    0.001    0.000 series.py:640(name)
      163    0.000    0.000    0.000    0.000 blocks.py:2057(check_ndim)
      163    0.000    0.000    0.000    0.000 blocks.py:1962(maybe_coerce_values)
      339    0.000    0.000    0.000    0.000 inference.py:321(is_hashable)
      564    0.000    0.000    0.000    0.000 generic.py:43(_check)
      163    0.000    0.000    0.001    0.000 config.py:127(_get_option)
  920/910    0.000    0.000    0.002    0.000 {built-in method builtins.getattr}
        1    0.000    0.000    0.040    0.040 869757136.py:2(find_average_b)
        8    0.000    0.000    0.013    0.002 blocks.py:1116(take_nd)
      163    0.000    0.000    0.000    0.000 config.py:589(_get_root)
      326    0.000    0.000    0.000    0.000 config.py:603(_get_deprecated_option)
      171    0.000    0.000    0.000    0.000 series.py:590(name)
      159    0.000    0.000    0.001    0.000 base.py:5092(_can_hold_identifiers_and_holds_name)
      163    0.000    0.000    0.001    0.000 common.py:552(require_length_match)
      159    0.000    0.000    0.001    0.000 base.py:3585(get_loc)
      180    0.000    0.000    0.000    0.000 common.py:1587(_is_dtype_type)
      163    0.000    0.000    0.000    0.000 construction.py:630(_sanitize_ndim)
        2    0.000    0.000    0.013    0.007 managers.py:689(<listcomp>)
        2    0.000    0.000    0.000    0.000 datetimelike.py:836(_isnan)
     10/8    0.000    0.000    0.013    0.002 take.py:57(take_nd)
      165    0.000    0.000    0.000    0.000 flags.py:47(__init__)
        1    0.000    0.000    0.005    0.005 managers.py:1551(as_array)
      163    0.000    0.000    0.000    0.000 indexing.py:2486(check_deprecated_indexers)
      163    0.000    0.000    0.001    0.000 config.py:255(__call__)
        8    0.000    0.000    0.000    0.000 take.py:326(_get_take_nd_function)
        1    0.000    0.000    0.000    0.000 base.py:744(__iter__)
        2    0.000    0.000    0.001    0.000 datetimes.py:2019(_sequence_to_dt64ns)
      159    0.000    0.000    0.001    0.000 base.py:5660(_get_values_for_loc)
      159    0.000    0.000    0.000    0.000 base.py:6298(_maybe_cast_indexer)
        2    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 frame.py:3463(__getitem__)
      163    0.000    0.000    0.000    0.000 managers.py:1714(__init__)
      169    0.000    0.000    0.000    0.000 common.py:1721(validate_all_hashable)
      167    0.000    0.000    0.001    0.000 construction.py:379(extract_array)
        8    0.000    0.000    0.012    0.002 take.py:120(_take_nd_ndarray)
      159    0.000    0.000    0.000    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
        1    0.000    0.000    0.000    0.000 cast.py:1789(find_common_type)
      171    0.000    0.000    0.000    0.000 {built-in method builtins.all}
      163    0.000    0.000    0.000    0.000 construction.py:805(is_empty_data)
      338    0.000    0.000    0.000    0.000 common.py:1740(<genexpr>)
      501    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_list_like}
      167    0.000    0.000    0.000    0.000 base.py:7099(maybe_extract_name)
        2    0.000    0.000    0.005    0.003 accessors.py:79(_delegate_property_get)
        1    0.000    0.000    0.001    0.001 datetimes.py:645(astype)
      159    0.000    0.000    0.001    0.000 base.py:2440(is_object)
      165    0.000    0.000    0.000    0.000 series.py:687(_values)
      169    0.000    0.000    0.000    0.000 construction.py:438(ensure_wrapped_if_datetimelike)
      161    0.000    0.000    0.000    0.000 {built-in method numpy.array}
      163    0.000    0.000    0.000    0.000 config.py:642(_warn_if_deprecated)
        2    0.000    0.000    0.001    0.001 datetimes.py:115(f)
        2    0.000    0.000    0.000    0.000 base.py:1106(take)
        6    0.000    0.000    0.000    0.000 generic.py:5517(__finalize__)
      500    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
      163    0.000    0.000    0.000    0.000 construction.py:667(_sanitize_str_dtypes)
      525    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
        2    0.000    0.000    0.000    0.000 numerictypes.py:575(_can_coerce_all)
        2    0.000    0.000    0.000    0.000 datetimes.py:496(_unbox_scalar)
        9    0.000    0.000    0.000    0.000 {method 'view' of 'numpy.ndarray' objects}
      159    0.000    0.000    0.000    0.000 common.py:160(cast_scalar_indexer)
        2    0.000    0.000    0.000    0.000 numeric.py:2385(array_equal)
      163    0.000    0.000    0.000    0.000 series.py:542(_set_axis)
      163    0.000    0.000    0.000    0.000 construction.py:687(_maybe_repeat)
        2    0.000    0.000    0.000    0.000 utils.py:249(maybe_convert_indices)
       10    0.000    0.000    0.000    0.000 common.py:1274(is_bool_dtype)
        8    0.000    0.000    0.000    0.000 {pandas._libs.lib.item_from_zerodim}
        2    0.000    0.000    0.000    0.000 datetimelike.py:1911(maybe_infer_freq)
    33/29    0.000    0.000    0.000    0.000 {built-in method numpy.asarray}
       10    0.000    0.000    0.000    0.000 missing.py:150(_isna)
      163    0.000    0.000    0.000    0.000 common.py:346(apply_if_callable)
      163    0.000    0.000    0.000    0.000 config.py:630(_translate_key)
      163    0.000    0.000    0.000    0.000 common.py:1416(is_1d_only_ea_dtype)
        2    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis1_float64_float64}
        2    0.000    0.000    0.001    0.000 base.py:397(__new__)
      159    0.000    0.000    0.000    0.000 generic.py:636(_info_axis)
       18    0.000    0.000    0.000    0.000 managers.py:1851(dtype)
      172    0.000    0.000    0.000    0.000 common.py:147(<lambda>)
      159    0.000    0.000    0.000    0.000 base.py:2352(is_floating)
        1    0.000    0.000    0.000    0.000 datetimelike.py:403(astype)
        2    0.000    0.000    0.001    0.000 datetimes.py:305(__new__)
       12    0.000    0.000    0.000    0.000 common.py:581(is_dtype_equal)
        4    0.000    0.000    0.001    0.000 algorithms.py:1356(take)
      163    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'all' of 'numpy.ndarray' objects}
      193    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
      318    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_integer}
        3    0.000    0.000    0.000    0.000 dtypes.py:955(is_dtype)
        4    0.000    0.000    0.000    0.000 _ufunc_config.py:32(seterr)
      163    0.000    0.000    0.000    0.000 config.py:571(_select_options)
        8    0.000    0.000    0.000    0.000 blocks.py:267(make_block_same_class)
      191    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_scalar}
        2    0.000    0.000    0.000    0.000 take.py:350(wrapper)
       12    0.000    0.000    0.000    0.000 cast.py:468(maybe_promote)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
      172    0.000    0.000    0.000    0.000 common.py:145(classes)
      163    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
        2    0.000    0.000    0.000    0.000 series.py:825(__array__)
        2    0.000    0.000    0.000    0.000 {method 'take' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 base.py:5192(equals)
        2    0.000    0.000    0.000    0.000 base.py:554(_dtype_to_subclass)
        8    0.000    0.000    0.000    0.000 missing.py:626(is_valid_na_for_dtype)
      163    0.000    0.000    0.000    0.000 managers.py:1806(_block)
        1    0.000    0.000    0.040    0.040 {built-in method builtins.exec}
        4    0.000    0.000    0.000    0.000 common.py:1747(pandas_dtype)
        2    0.000    0.000    0.001    0.000 series.py:5613(_cmp_method)
        8    0.000    0.000    0.000    0.000 take.py:554(_take_preprocess_indexer_and_fill_value)
        2    0.000    0.000    0.000    0.000 array_ops.py:227(comparison_op)
        7    0.000    0.000    0.000    0.000 base.py:286(is_dtype)
        6    0.000    0.000    0.000    0.000 missing.py:572(na_value_for_dtype)
        2    0.000    0.000    0.000    0.000 datetimes.py:326(_simple_new)
        2    0.000    0.000    0.000    0.000 {built-in method _operator.eq}
      328    0.000    0.000    0.000    0.000 typing.py:1375(cast)
        2    0.000    0.000    0.000    0.000 datetimelike.py:850(_maybe_mask_results)
        2    0.000    0.000    0.001    0.000 _mixins.py:147(take)
       37    0.000    0.000    0.000    0.000 common.py:1552(get_dtype)
        2    0.000    0.000    0.014    0.007 generic.py:3609(take)
       10    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_platform_int}
        6    0.000    0.000    0.000    0.000 base.py:654(_simple_new)
       37    0.000    0.000    0.000    0.000 datetimes.py:545(dtype)
        2    0.000    0.000    0.013    0.007 managers.py:634(reindex_indexer)
        4    0.000    0.000    0.000    0.000 common.py:1240(is_float_dtype)
        2    0.000    0.000    0.000    0.000 expressions.py:63(_evaluate_standard)
        3    0.000    0.000    0.000    0.000 generic.py:5632(_protect_consolidate)
        6    0.000    0.000    0.000    0.000 common.py:1429(is_extension_array_dtype)
        8    0.000    0.000    0.000    0.000 {method 'any' of 'numpy.ndarray' objects}
       10    0.000    0.000    0.000    0.000 {built-in method pandas._libs.missing.checknull}
        2    0.000    0.000    0.001    0.000 accessors.py:61(_get_values)
        2    0.000    0.000    0.000    0.000 utils.py:197(validate_indices)
        2    0.000    0.000    0.000    0.000 common.py:497(is_categorical_dtype)
        2    0.000    0.000    0.000    0.000 common.py:680(is_integer_dtype)
        8    0.000    0.000    0.000    0.000 generic.py:560(_get_axis)
        2    0.000    0.000    0.014    0.007 managers.py:872(take)
        2    0.000    0.000    0.000    0.000 array_ops.py:134(_na_arithmetic_op)
        1    0.000    0.000    0.001    0.001 blocks.py:1463(get_values)
        2    0.000    0.000    0.001    0.000 datetimes.py:341(_from_sequence_not_strict)
        1    0.000    0.000    0.005    0.005 frame.py:10817(values)
      159    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_object}
        2    0.000    0.000    0.014    0.007 generic.py:3708(_take_with_is_copy)
        2    0.000    0.000    0.002    0.001 extension.py:67(fget)
        2    0.000    0.000    0.000    0.000 {built-in method pandas._libs.tslibs.timezones.maybe_get_tz}
        8    0.000    0.000    0.000    0.000 inference.py:184(is_array_like)
       19    0.000    0.000    0.000    0.000 numerictypes.py:584(<listcomp>)
      159    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_float}
        2    0.000    0.000    0.000    0.000 datetimelike.py:843(_hasna)
        2    0.000    0.000    0.000    0.000 numeric.py:139(_ensure_array)
        6    0.000    0.000    0.000    0.000 cast.py:515(_maybe_promote)
        3    0.000    0.000    0.000    0.000 generic.py:5646(_consolidate_inplace)
        2    0.000    0.000    0.000    0.000 common.py:786(is_unsigned_integer_dtype)
        2    0.000    0.000    0.000    0.000 series.py:2988(_construct_result)
        2    0.000    0.000    0.000    0.000 datetimelike.py:591(_validate_scalar)
        2    0.000    0.000    0.015    0.007 indexing.py:954(__getitem__)
        3    0.000    0.000    0.000    0.000 common.py:423(is_period_dtype)
        1    0.000    0.000    0.000    0.000 numerictypes.py:599(find_common_type)
        2    0.000    0.000    0.000    0.000 datetimes.py:2268(maybe_convert_dtype)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:425(__init__)
        2    0.000    0.000    0.015    0.007 indexing.py:1169(_getitem_axis)
        4    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 common.py:105(is_bool_indexer)
        2    0.000    0.000    0.000    0.000 managers.py:970(from_blocks)
        2    0.000    0.000    0.000    0.000 frame.py:3923(_get_item_cache)
        3    0.000    0.000    0.003    0.001 blocks.py:222(get_values)
        2    0.000    0.000    0.000    0.000 indexing.py:267(loc)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(array_equal)
        8    0.000    0.000    0.000    0.000 common.py:155(<lambda>)
        2    0.000    0.000    0.005    0.003 accessor.py:79(_getter)
        2    0.000    0.000    0.000    0.000 utils.py:457(check_array_indexer)
        2    0.000    0.000    0.015    0.007 indexing.py:981(_getbool_axis)
        2    0.000    0.000    0.000    0.000 expressions.py:92(_evaluate_numexpr)
        7    0.000    0.000    0.000    0.000 common.py:315(is_datetime64_dtype)
        2    0.000    0.000    0.000    0.000 expressions.py:223(evaluate)
        2    0.000    0.000    0.000    0.000 datetimes.py:795(_local_timestamps)
        4    0.000    0.000    0.000    0.000 common.py:1483(is_ea_or_datetimelike_dtype)
        1    0.000    0.000    0.000    0.000 managers.py:1689(<listcomp>)
        4    0.000    0.000    0.000    0.000 _ufunc_config.py:131(geterr)
       18    0.000    0.000    0.000    0.000 series.py:575(dtype)
        6    0.000    0.000    0.000    0.000 common.py:1148(needs_i8_conversion)
        4    0.000    0.000    0.000    0.000 base.py:53(shape)
        6    0.000    0.000    0.000    0.000 flags.py:51(allows_duplicate_labels)
        5    0.000    0.000    0.000    0.000 datetimelike.py:280(asi8)
        8    0.000    0.000    0.000    0.000 _methods.py:54(_any)
        4    0.000    0.000    0.000    0.000 datetimes.py:570(tz)
        2    0.000    0.000    0.000    0.000 indexing.py:2357(check_bool_indexer)
        2    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.001    0.000 arraylike.py:38(__eq__)
       11    0.000    0.000    0.000    0.000 base.py:4834(_values)
        2    0.000    0.000    0.000    0.000 {built-in method pandas._libs.tslibs.offsets.to_offset}
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.dtypes_all_equal}
       10    0.000    0.000    0.000    0.000 missing.py:67(isna)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:429(__enter__)
        2    0.000    0.000    0.000    0.000 cast.py:1830(<genexpr>)
        6    0.000    0.000    0.000    0.000 flags.py:83(allows_duplicate_labels)
        2    0.000    0.000    0.000    0.000 common.py:1399(is_1d_only_ea_obj)
        2    0.000    0.000    0.001    0.000 common.py:55(new_method)
        2    0.000    0.000    0.000    0.000 missing.py:138(dispatch_fill_zeros)
        2    0.000    0.000    0.000    0.000 common.py:191(is_sparse)
       10    0.000    0.000    0.000    0.000 generic.py:546(_get_axis_number)
        1    0.000    0.000    0.000    0.000 numerictypes.py:651(<listcomp>)
        4    0.000    0.000    0.000    0.000 {built-in method numpy.seterrobj}
        6    0.000    0.000    0.000    0.000 base.py:834(_reset_identity)
        2    0.000    0.000    0.000    0.000 frame.py:587(__init__)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        3    0.000    0.000    0.000    0.000 generic.py:5650(f)
        2    0.000    0.000    0.000    0.000 numeric.py:319(_assert_safe_casting)
        8    0.000    0.000    0.000    0.000 {built-in method numpy.geterrobj}
        2    0.000    0.000    0.000    0.000 {built-in method _simple_new}
        6    0.000    0.000    0.000    0.000 generic.py:328(attrs)
        1    0.000    0.000    0.040    0.040 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 managers.py:1683(_consolidate_check)
        6    0.000    0.000    0.000    0.000 cast.py:454(ensure_dtype_can_hold_na)
        6    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x00007FFE9B8EDC60}
        2    0.000    0.000    0.000    0.000 common.py:540(is_string_dtype)
        4    0.000    0.000    0.000    0.000 cast.py:1711(sanitize_to_nanoseconds)
       12    0.000    0.000    0.000    0.000 base.py:55(<genexpr>)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.any}
        6    0.000    0.000    0.000    0.000 blocks.py:354(dtype)
        3    0.000    0.000    0.000    0.000 managers.py:618(consolidate)
        4    0.000    0.000    0.000    0.000 common.py:732(is_signed_integer_dtype)
        2    0.000    0.000    0.000    0.000 common.py:1042(is_numeric_v_string_like)
        5    0.000    0.000    0.000    0.000 cast.py:1835(<genexpr>)
        2    0.000    0.000    0.000    0.000 generic.py:3925(_set_is_copy)
        3    0.000    0.000    0.000    0.000 managers.py:1675(is_consolidated)
        2    0.000    0.000    0.000    0.000 missing.py:391(array_equivalent)
        1    0.000    0.000    0.000    0.000 managers.py:1625(<listcomp>)
        2    0.000    0.000    0.000    0.000 {method '_from_backing_data' of 'pandas._libs.arrays.NDArrayBacked' objects}
        3    0.000    0.000    0.000    0.000 common.py:389(is_timedelta64_dtype)
        3    0.000    0.000    0.000    0.000 cast.py:1828(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:210(interleaved_dtype)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:434(__exit__)
        2    0.000    0.000    0.000    0.000 expressions.py:72(_can_use_numexpr)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        2    0.000    0.000    0.000    0.000 {method 'min' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 numeric.py:188(_validate_dtype)
        1    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 base.py:803(is_)
        3    0.000    0.000    0.000    0.000 datetimelike.py:880(freq)
        2    0.000    0.000    0.000    0.000 generic.py:566(_get_block_manager_axis)
        2    0.000    0.000    0.000    0.000 common.py:348(is_datetime64tz_dtype)
        1    0.000    0.000    0.000    0.000 {built-in method fromkeys}
       16    0.000    0.000    0.000    0.000 base.py:326(ndim)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.sum}
        8    0.000    0.000    0.000    0.000 common.py:150(classes_and_not_datetimelike)
        1    0.000    0.000    0.000    0.000 common.py:925(is_datetime64_ns_dtype)
        5    0.000    0.000    0.000    0.000 cast.py:1819(<genexpr>)
        2    0.000    0.000    0.000    0.000 common.py:75(get_op_result_name)
        2    0.000    0.000    0.000    0.000 base.py:945(dtype)
       12    0.000    0.000    0.000    0.000 generic.py:349(flags)
        2    0.000    0.000    0.000    0.000 _methods.py:60(_all)
        2    0.000    0.000    0.000    0.000 dispatch.py:11(should_extension_dispatch)
        2    0.000    0.000    0.000    0.000 {method 'max' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 common.py:1528(_is_dtype)
        2    0.000    0.000    0.000    0.000 frame.py:578(_constructor)
        4    0.000    0.000    0.000    0.000 blocks.py:244(mgr_locs)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.asanyarray}
        2    0.000    0.000    0.000    0.000 _methods.py:42(_amin)
        4    0.000    0.000    0.000    0.000 base.py:1658(name)
        2    0.000    0.000    0.000    0.000 function.py:49(__call__)
        2    0.000    0.000    0.000    0.000 common.py:569(condition)
        1    0.000    0.000    0.000    0.000 base.py:465(find)
        2    0.000    0.000    0.000    0.000 base.py:890(__array__)
        2    0.000    0.000    0.000    0.000 base.py:1132(_maybe_disallow_fill)
        2    0.000    0.000    0.000    0.000 datetimes.py:96(tz_to_dtype)
        2    0.000    0.000    0.000    0.000 datetimelike.py:1876(validate_inferred_freq)
        1    0.000    0.000    0.000    0.000 frame.py:821(shape)
        2    0.000    0.000    0.000    0.000 managers.py:156(blknos)
        2    0.000    0.000    0.000    0.000 _methods.py:38(_amax)
        2    0.000    0.000    0.000    0.000 datetimes.py:505(_check_compatible_with)
        2    0.000    0.000    0.000    0.000 numeric.py:199(_ensure_dtype)
        2    0.000    0.000    0.000    0.000 numeric.py:2381(_array_equal_dispatcher)
        2    0.000    0.000    0.000    0.000 series.py:523(_constructor)
        4    0.000    0.000    0.000    0.000 datetimes.py:2401(validate_tz_from_dtype)
        4    0.000    0.000    0.000    0.000 numeric.py:331(_is_all_dates)
        2    0.000    0.000    0.000    0.000 managers.py:172(blklocs)
        1    0.000    0.000    0.000    0.000 {method 'transpose' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 datetimes.py:2359(_validate_dt64_dtype)
        1    0.000    0.000    0.000    0.000 blocks.py:350(shape)
        4    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_iterator}
        1    0.000    0.000    0.000    0.000 managers.py:217(is_single_block)
        2    0.000    0.000    0.000    0.000 managers.py:916(__init__)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        2    0.000    0.000    0.000    0.000 base.py:229(disallow_kwargs)
        1    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 numerictypes.py:652(<listcomp>)

**Вариант C**

1159 function calls (1144 primitive calls) in 0.105 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.013    0.013    0.013    0.013 {pandas._libs.algos.take_2d_axis1_object_object}
        1    0.007    0.007    0.013    0.013 datetimes.py:2019(_sequence_to_dt64ns)
        1    0.006    0.006    0.105    0.105 3013737422.py:2(find_average_c)
        2    0.004    0.002    0.011    0.006 frame.py:3463(__getitem__)
        1    0.004    0.004    0.004    0.004 datetimelike.py:836(_isnan)
        3    0.004    0.001    0.004    0.001 base.py:4987(__contains__)
        8    0.003    0.000    0.003    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.003    0.003    0.003    0.003 {pandas._libs.tslibs.fields.get_date_field}
        1    0.002    0.002    0.024    0.024 accessors.py:61(_get_values)
        4    0.002    0.000    0.002    0.000 {method 'view' of 'numpy.ndarray' objects}
        1    0.002    0.002    0.019    0.019 datetimes.py:305(__new__)
        1    0.002    0.002    0.002    0.002 datetimelike.py:1911(maybe_infer_freq)
        1    0.002    0.002    0.003    0.003 array_ops.py:227(comparison_op)
        1    0.001    0.001    0.051    0.051 accessors.py:79(_delegate_property_get)
        2    0.001    0.001    0.001    0.001 {pandas._libs.algos.take_2d_axis1_int64_int64}
        2    0.001    0.001    0.003    0.001 frame.py:3923(_get_item_cache)
        1    0.001    0.001    0.017    0.017 datetimes.py:341(_from_sequence_not_strict)
        2    0.001    0.001    0.001    0.001 construction.py:698(_try_cast)
        1    0.001    0.001    0.001    0.001 indexing.py:267(loc)
        1    0.001    0.001    0.012    0.012 datetimes.py:115(f)
        2    0.001    0.001    0.002    0.001 construction.py:470(sanitize_array)
        1    0.001    0.001    0.001    0.001 datetimes.py:496(_unbox_scalar)
        2    0.001    0.001    0.001    0.001 blocks.py:2043(new_block)
        1    0.001    0.001    0.001    0.001 dtypes.py:955(is_dtype)
       16    0.001    0.000    0.001    0.000 datetimes.py:545(dtype)
        1    0.001    0.001    0.001    0.001 {built-in method bottleneck.reduce.nanmean}
        1    0.001    0.001    0.002    0.002 take.py:350(wrapper)
        3    0.001    0.000    0.007    0.002 series.py:323(__init__)
        1    0.001    0.001    0.004    0.004 generic.py:10601(_stat_function)
        1    0.001    0.001    0.001    0.001 base.py:1106(take)
        4    0.001    0.000    0.002    0.001 {method 'any' of 'numpy.ndarray' objects}
      241    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
        5    0.001    0.000    0.001    0.000 {built-in method pandas._libs.missing.checknull}
    58/53    0.001    0.000    0.017    0.000 {built-in method builtins.getattr}
    16/14    0.001    0.000    0.001    0.000 {built-in method numpy.asarray}
        9    0.001    0.000    0.001    0.000 managers.py:1851(dtype)
        1    0.001    0.001    0.002    0.002 nanops.py:119(f)
        1    0.001    0.001    0.004    0.004 base.py:397(__new__)
        1    0.001    0.001    0.001    0.001 {built-in method pandas._libs.tslibs.offsets.to_offset}
        9    0.001    0.000    0.001    0.000 series.py:575(dtype)
        1    0.001    0.001    0.022    0.022 managers.py:689(<listcomp>)
        3    0.001    0.000    0.001    0.000 base.py:654(_simple_new)
        2    0.001    0.000    0.001    0.000 common.py:1240(is_float_dtype)
        1    0.001    0.001    0.001    0.001 utils.py:249(maybe_convert_indices)
        1    0.001    0.001    0.001    0.001 base.py:554(_dtype_to_subclass)
        3    0.001    0.000    0.001    0.000 missing.py:572(na_value_for_dtype)
        1    0.000    0.000    0.000    0.000 {pandas._libs.algos.take_2d_axis1_float64_float64}
       12    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_list_like}
        2    0.000    0.000    0.001    0.000 config.py:109(_get_single_key)
        4    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
        1    0.000    0.000    0.001    0.001 numeric.py:2385(array_equal)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1033(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
        2    0.000    0.000    0.001    0.000 common.py:315(is_datetime64_dtype)
        1    0.000    0.000    0.004    0.004 series.py:5613(_cmp_method)
        1    0.000    0.000    0.001    0.001 nanops.py:178(_has_infs)
        4    0.000    0.000    0.021    0.005 blocks.py:1116(take_nd)
        3    0.000    0.000    0.001    0.000 cast.py:515(_maybe_promote)
      5/4    0.000    0.000    0.021    0.005 take.py:57(take_nd)
        1    0.000    0.000    0.003    0.003 series.py:4435(_reduce)
        5    0.000    0.000    0.000    0.000 {pandas._libs.lib.item_from_zerodim}
        1    0.000    0.000    0.051    0.051 accessor.py:79(_getter)
        3    0.000    0.000    0.001    0.000 common.py:1274(is_bool_dtype)
        1    0.000    0.000    0.001    0.001 base.py:3585(get_loc)
        1    0.000    0.000    0.002    0.002 numeric.py:139(_ensure_array)
        3    0.000    0.000    0.000    0.000 nanops.py:86(<genexpr>)
        1    0.000    0.000    0.001    0.001 utils.py:197(validate_indices)
        4    0.000    0.000    0.000    0.000 take.py:326(_get_take_nd_function)
        4    0.000    0.000    0.000    0.000 common.py:155(<lambda>)
       16    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_scalar}
        1    0.000    0.000    0.017    0.017 extension.py:67(fget)
        1    0.000    0.000    0.001    0.001 array_ops.py:134(_na_arithmetic_op)
        1    0.000    0.000    0.000    0.000 {built-in method pandas._libs.tslibs.timezones.maybe_get_tz}
        2    0.000    0.000    0.000    0.000 base.py:7004(ensure_index)
        2    0.000    0.000    0.000    0.000 common.py:1483(is_ea_or_datetimelike_dtype)
        4    0.000    0.000    0.000    0.000 generic.py:239(__init__)
        1    0.000    0.000    0.003    0.003 nanops.py:83(_f)
       14    0.000    0.000    0.000    0.000 common.py:1552(get_dtype)
       12    0.000    0.000    0.001    0.000 common.py:1587(_is_dtype_type)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(array_equal)
        1    0.000    0.000    0.024    0.024 managers.py:872(take)
       11    0.000    0.000    0.000    0.000 inference.py:321(is_hashable)
        1    0.000    0.000    0.000    0.000 common.py:191(is_sparse)
        2    0.000    0.000    0.000    0.000 {method 'copy' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:105(is_bool_indexer)
        1    0.000    0.000    0.000    0.000 common.py:389(is_timedelta64_dtype)
        1    0.000    0.000    0.001    0.001 common.py:423(is_period_dtype)
        1    0.000    0.000    0.002    0.002 datetimes.py:795(_local_timestamps)
        1    0.000    0.000    0.000    0.000 {method 'nonzero' of 'numpy.ndarray' objects}
        5    0.000    0.000    0.002    0.000 generic.py:5561(__getattr__)
        2    0.000    0.000    0.002    0.001 datetimelike.py:280(asi8)
        1    0.000    0.000    0.004    0.004 generic.py:10686(mean)
        1    0.000    0.000    0.005    0.005 generic.py:11106(mean)
        5    0.000    0.000    0.000    0.000 series.py:640(name)
        1    0.000    0.000    0.000    0.000 {method 'take' of 'numpy.ndarray' objects}
        6    0.000    0.000    0.001    0.000 cast.py:468(maybe_promote)
        6    0.000    0.000    0.000    0.000 common.py:161(is_object_dtype)
        2    0.000    0.000    0.000    0.000 base.py:5037(__getitem__)
        1    0.000    0.000    0.000    0.000 indexing.py:2357(check_bool_indexer)
        5    0.000    0.000    0.000    0.000 common.py:581(is_dtype_equal)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        3    0.000    0.000    0.000    0.000 base.py:286(is_dtype)
        4    0.000    0.000    0.000    0.000 managers.py:1862(internal_values)
        5    0.000    0.000    0.000    0.000 common.py:1721(validate_all_hashable)
        1    0.000    0.000    0.000    0.000 datetimes.py:96(tz_to_dtype)
        1    0.000    0.000    0.000    0.000 series.py:1238(_set_as_cached)
        4    0.000    0.000    0.000    0.000 base.py:7099(maybe_extract_name)
        3    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x00007FFE9B8EDC60}
        2    0.000    0.000    0.000    0.000 construction.py:667(_sanitize_str_dtypes)
        5    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_platform_int}
        7    0.000    0.000    0.000    0.000 base.py:884(__len__)
        1    0.000    0.000    0.000    0.000 common.py:680(is_integer_dtype)
        1    0.000    0.000    0.000    0.000 {method 'any' of 'numpy.generic' objects}
        1    0.000    0.000    0.000    0.000 base.py:890(__array__)
        6    0.000    0.000    0.000    0.000 series.py:590(name)
        1    0.000    0.000    0.006    0.006 datetimelike.py:850(_maybe_mask_results)
        2    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_integer}
        2    0.000    0.000    0.000    0.000 construction.py:805(is_empty_data)
        3    0.000    0.000    0.000    0.000 common.py:1429(is_extension_array_dtype)
        2    0.000    0.000    0.000    0.000 common.py:552(require_length_match)
        2    0.000    0.000    0.000    0.000 blocks.py:2057(check_ndim)
        1    0.000    0.000    0.001    0.001 datetimelike.py:591(_validate_scalar)
        2    0.000    0.000    0.000    0.000 blocks.py:1962(maybe_coerce_values)
        1    0.000    0.000    0.004    0.004 datetimes.py:2268(maybe_convert_dtype)
        3    0.000    0.000    0.000    0.000 common.py:346(apply_if_callable)
        1    0.000    0.000    0.000    0.000 {built-in method _simple_new}
        1    0.000    0.000    0.001    0.001 numeric.py:319(_assert_safe_casting)
        1    0.000    0.000    0.000    0.000 expressions.py:92(_evaluate_numexpr)
        2    0.000    0.000    0.002    0.001 managers.py:1745(from_array)
       12    0.000    0.000    0.000    0.000 {built-in method numpy.geterrobj}
        1    0.000    0.000    0.004    0.004 arraylike.py:38(__eq__)
        4    0.000    0.000    0.000    0.000 missing.py:626(is_valid_na_for_dtype)
        2    0.000    0.000    0.004    0.002 algorithms.py:1356(take)
        1    0.000    0.000    0.000    0.000 datetimelike.py:880(freq)
        1    0.000    0.000    0.000    0.000 nanops.py:162(_bn_ok_dtype)
        1    0.000    0.000    0.000    0.000 utils.py:457(check_array_indexer)
        1    0.000    0.000    0.000    0.000 datetimelike.py:1876(validate_inferred_freq)
        4    0.000    0.000    0.017    0.004 take.py:120(_take_nd_ndarray)
        1    0.000    0.000    0.000    0.000 numeric.py:2381(_array_equal_dispatcher)
        1    0.000    0.000    0.025    0.025 generic.py:3609(take)
        8    0.000    0.000    0.001    0.000 generic.py:5577(__setattr__)
        1    0.000    0.000    0.000    0.000 expressions.py:223(evaluate)
        4    0.000    0.000    0.000    0.000 generic.py:5517(__finalize__)
        5    0.000    0.000    0.001    0.000 missing.py:150(_isna)
        6    0.000    0.000    0.000    0.000 _ufunc_config.py:32(seterr)
        6    0.000    0.000    0.000    0.000 _ufunc_config.py:131(geterr)
       40    0.000    0.000    0.000    0.000 generic.py:43(_check)
        4    0.000    0.000    0.000    0.000 construction.py:379(extract_array)
        3    0.000    0.000    0.000    0.000 indexing.py:2486(check_deprecated_indexers)
        1    0.000    0.000    0.026    0.026 indexing.py:954(__getitem__)
        1    0.000    0.000    0.022    0.022 managers.py:634(reindex_indexer)
        1    0.000    0.000    0.105    0.105 {built-in method builtins.exec}
        5    0.000    0.000    0.002    0.000 _methods.py:54(_any)
        4    0.000    0.000    0.000    0.000 flags.py:47(__init__)
        3    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 base.py:5192(equals)
        1    0.000    0.000    0.000    0.000 series.py:2988(_construct_result)
        2    0.000    0.000    0.001    0.000 config.py:255(__call__)
        1    0.000    0.000    0.000    0.000 {built-in method _operator.eq}
        1    0.000    0.000    0.005    0.005 _mixins.py:147(take)
        2    0.000    0.000    0.000    0.000 config.py:642(_warn_if_deprecated)
        1    0.000    0.000    0.004    0.004 common.py:55(new_method)
        1    0.000    0.000    0.000    0.000 common.py:497(is_categorical_dtype)
       19    0.000    0.000    0.001    0.000 {built-in method builtins.hasattr}
       24    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
        4    0.000    0.000    0.000    0.000 take.py:554(_take_preprocess_indexer_and_fill_value)
        1    0.000    0.000    0.026    0.026 indexing.py:1169(_getitem_axis)
        1    0.000    0.000    0.000    0.000 managers.py:1026(iget)
        4    0.000    0.000    0.000    0.000 blocks.py:267(make_block_same_class)
        4    0.000    0.000    0.000    0.000 generic.py:560(_get_axis)
        1    0.000    0.000    0.000    0.000 datetimes.py:326(_simple_new)
        1    0.000    0.000    0.006    0.006 datetimelike.py:843(_hasna)
        1    0.000    0.000    0.001    0.001 frame.py:3411(_ixs)
        6    0.000    0.000    0.000    0.000 {built-in method numpy.seterrobj}
        2    0.000    0.000    0.000    0.000 blocks.py:1991(get_block_type)
        2    0.000    0.000    0.000    0.000 series.py:542(_set_axis)
        4    0.000    0.000    0.000    0.000 series.py:687(_values)
        6    0.000    0.000    0.000    0.000 generic.py:546(_get_axis_number)
        4    0.000    0.000    0.000    0.000 flags.py:83(allows_duplicate_labels)
    38/31    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        4    0.000    0.000    0.000    0.000 config.py:603(_get_deprecated_option)
        1    0.000    0.000    0.000    0.000 managers.py:970(from_blocks)
        3    0.000    0.000    0.000    0.000 common.py:160(cast_scalar_indexer)
        3    0.000    0.000    0.000    0.000 _ufunc_config.py:425(__init__)
        1    0.000    0.000    0.025    0.025 indexing.py:981(_getbool_axis)
        8    0.000    0.000    0.000    0.000 common.py:147(<lambda>)
        2    0.000    0.000    0.001    0.000 config.py:127(_get_option)
        3    0.000    0.000    0.000    0.000 _ufunc_config.py:429(__enter__)
        3    0.000    0.000    0.000    0.000 managers.py:1714(__init__)
        2    0.000    0.000    0.000    0.000 config.py:571(_select_options)
        1    0.000    0.000    0.000    0.000 base.py:2440(is_object)
        1    0.000    0.000    0.000    0.000 frame.py:587(__init__)
        1    0.000    0.000    0.000    0.000 generic.py:5650(f)
        4    0.000    0.000    0.000    0.000 inference.py:184(is_array_like)
        1    0.000    0.000    0.000    0.000 {method 'all' of 'numpy.ndarray' objects}
        5    0.000    0.000    0.001    0.000 missing.py:67(isna)
        2    0.000    0.000    0.000    0.000 construction.py:630(_sanitize_ndim)
        1    0.000    0.000    0.000    0.000 generic.py:5632(_protect_consolidate)
        3    0.000    0.000    0.000    0.000 _ufunc_config.py:434(__exit__)
        1    0.000    0.000    0.000    0.000 managers.py:618(consolidate)
        1    0.000    0.000    0.000    0.000 generic.py:5646(_consolidate_inplace)
        1    0.000    0.000    0.001    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 base.py:5092(_can_hold_identifiers_and_holds_name)
        1    0.000    0.000    0.000    0.000 common.py:1042(is_numeric_v_string_like)
        1    0.000    0.000    0.000    0.000 base.py:53(shape)
        2    0.000    0.000    0.000    0.000 config.py:589(_get_root)
        1    0.000    0.000    0.000    0.000 {method '_from_backing_data' of 'pandas._libs.arrays.NDArrayBacked' objects}
        1    0.000    0.000    0.000    0.000 common.py:540(is_string_dtype)
        2    0.000    0.000    0.000    0.000 common.py:1416(is_1d_only_ea_dtype)
        1    0.000    0.000    0.000    0.000 expressions.py:63(_evaluate_standard)
        1    0.000    0.000    0.000    0.000 frame.py:578(_constructor)
        8    0.000    0.000    0.000    0.000 common.py:145(classes)
        1    0.000    0.000    0.000    0.000 common.py:786(is_unsigned_integer_dtype)
        2    0.000    0.000    0.000    0.000 numeric.py:188(_validate_dtype)
        1    0.000    0.000    0.001    0.001 {method 'min' of 'numpy.ndarray' objects}
        4    0.000    0.000    0.000    0.000 flags.py:51(allows_duplicate_labels)
        1    0.000    0.000    0.025    0.025 generic.py:3708(_take_with_is_copy)
        3    0.000    0.000    0.000    0.000 managers.py:1806(_block)
        1    0.000    0.000    0.000    0.000 generic.py:636(_info_axis)
        2    0.000    0.000    0.000    0.000 cast.py:1711(sanitize_to_nanoseconds)
        1    0.000    0.000    0.000    0.000 dispatch.py:11(should_extension_dispatch)
        1    0.000    0.000    0.000    0.000 _validators.py:218(validate_bool_kwarg)
        1    0.000    0.000    0.000    0.000 base.py:2352(is_floating)
        1    0.000    0.000    0.000    0.000 datetimes.py:570(tz)
        4    0.000    0.000    0.000    0.000 base.py:4834(_values)
        1    0.000    0.000    0.000    0.000 datetimes.py:505(_check_compatible_with)
        1    0.000    0.000    0.000    0.000 datetimes.py:2359(_validate_dt64_dtype)
        1    0.000    0.000    0.000    0.000 frame.py:3906(_box_col_values)
       14    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
        1    0.000    0.000    0.000    0.000 missing.py:138(dispatch_fill_zeros)
        1    0.000    0.000    0.000    0.000 common.py:1399(is_1d_only_ea_obj)
        5    0.000    0.000    0.000    0.000 construction.py:438(ensure_wrapped_if_datetimelike)
        5    0.000    0.000    0.000    0.000 {built-in method builtins.all}
       10    0.000    0.000    0.000    0.000 common.py:1740(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:6298(_maybe_cast_indexer)
        4    0.000    0.000    0.000    0.000 common.py:1148(needs_i8_conversion)
        3    0.000    0.000    0.000    0.000 base.py:834(_reset_identity)
        4    0.000    0.000    0.000    0.000 generic.py:328(attrs)
        1    0.000    0.000    0.105    0.105 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 common.py:348(is_datetime64tz_dtype)
        2    0.000    0.000    0.000    0.000 common.py:732(is_signed_integer_dtype)
        1    0.000    0.000    0.000    0.000 generic.py:3925(_set_is_copy)
        1    0.000    0.000    0.000    0.000 expressions.py:72(_can_use_numexpr)
        2    0.000    0.000    0.000    0.000 function.py:49(__call__)
        1    0.000    0.000    0.000    0.000 missing.py:391(array_equivalent)
        2    0.000    0.000    0.000    0.000 nanops.py:79(check)
        1    0.000    0.000    0.000    0.000 managers.py:1675(is_consolidated)
        1    0.000    0.000    0.000    0.000 blocks.py:354(dtype)
        1    0.000    0.000    0.000    0.000 blocks.py:358(iget)
        3    0.000    0.000    0.000    0.000 cast.py:454(ensure_dtype_can_hold_na)
        2    0.000    0.000    0.000    0.000 base.py:1658(name)
        8    0.000    0.000    0.000    0.000 base.py:326(ndim)
        2    0.000    0.000    0.000    0.000 config.py:630(_translate_key)
        1    0.000    0.000    0.000    0.000 base.py:1132(_maybe_disallow_fill)
        1    0.000    0.000    0.000    0.000 common.py:75(get_op_result_name)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.any}
        1    0.000    0.000    0.000    0.000 common.py:1528(_is_dtype)
        2    0.000    0.000    0.000    0.000 base.py:803(is_)
        8    0.000    0.000    0.000    0.000 generic.py:349(flags)
        2    0.000    0.000    0.000    0.000 managers.py:172(blklocs)
        2    0.000    0.000    0.000    0.000 construction.py:687(_maybe_repeat)
        2    0.000    0.000    0.000    0.000 managers.py:156(blknos)
        2    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
        1    0.000    0.000    0.000    0.000 series.py:825(__array__)
        4    0.000    0.000    0.000    0.000 common.py:150(classes_and_not_datetimelike)
        1    0.000    0.000    0.000    0.000 _methods.py:60(_all)
        2    0.000    0.000    0.000    0.000 datetimes.py:2401(validate_tz_from_dtype)
        6    0.000    0.000    0.000    0.000 typing.py:1375(cast)
        1    0.000    0.000    0.000    0.000 common.py:569(condition)
        1    0.000    0.000    0.001    0.001 _methods.py:42(_amin)
        1    0.000    0.000    0.000    0.000 numeric.py:199(_ensure_dtype)
        1    0.000    0.000    0.000    0.000 {method 'max' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asanyarray}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
        1    0.000    0.000    0.000    0.000 generic.py:566(_get_block_manager_axis)
        3    0.000    0.000    0.000    0.000 base.py:55(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 common.py:1747(pandas_dtype)
        1    0.000    0.000    0.000    0.000 base.py:945(dtype)
        3    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_iterator}
        3    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_float}
        1    0.000    0.000    0.000    0.000 _methods.py:38(_amax)
        2    0.000    0.000    0.000    0.000 numeric.py:331(_is_all_dates)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 base.py:229(disallow_kwargs)
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_bool}
        1    0.000    0.000    0.000    0.000 series.py:523(_constructor)
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 managers.py:916(__init__)

Решение  A и B занимают больше времени из-за использования генераторов списка по методу df.iterrows(), следовательно, если отказаться от этого метода можно ускорить работу функции даже без модуля mean

In [111]:
"""Пример:"""
#B.sped_up
def find_average_b():
    rating = df.loc[df['submitted'].dt.year == 2010].rating.sum()
    average_rating = rating / df.loc[df['submitted'].dt.year == 2010].shape[0]
    return average_rating
%prun av1=find_average_b()
print(av1)
execution_time=timeit.timeit(find_average_b, number=1)
print(f'Время исполнения - {execution_time}')

 4.270440251572327
Время исполнения - 0.0035108000001855544


Как видно, представленное решение быстрее

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [112]:
df_fav = df.loc[df['submitted'].dt.year == 2010]

In [132]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%prun get_word_reviews_count(df_fav)

In [133]:
from collections import defaultdict
def get_word_reviews_count(df):
    word_reviews_count = defaultdict(list)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        unique_words = set(words)  # Используем множество для поиска уникальных слов
        
        for word in unique_words:
            word_reviews_count[word].append(row['recipe_id'])
    
    return word_reviews_count

%prun get_word_reviews_count(df_fav)

Чтобы ускорить работу кода используется множества и автоматическое создание пустого списка, чтобы слова были уникальными

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [128]:
import numba
import time

In [156]:
df_fav_100 = df.loc[df.rating != 0]

In [157]:
#A
def mape_a(df):
    ratings = df['rating'].values
    mean_rating = sum(ratings) / len(ratings)
    mape_sum = sum([abs((rating - mean_rating) / mean_rating) for rating in ratings])
    
    mape = mape_sum / len(ratings)
    return mape

%time mape_a(df_fav_100)

Wall time: 67.6 ms


0.11138533165746958

In [158]:
#B
# @numba.njit
def mape_b(df):
    ratings = df.rating.values
    n = len(ratings)
    mean_rating = 0.0
    mape_sum = 0.0
    mean_rating = sum([ratings[i] for i in range(n)])/n
    mape_sum = sum([abs((ratings[i] - mean_rating) / mean_rating) for i in range(n)])
    
    mape = mape_sum / n
    return mape

%time mape_b(df_fav_100)

Wall time: 15.4 ms


0.11138533165746958

In [159]:
#C
def mape_c(df):
    ratings = df.rating.values
    mean_rating = np.mean(ratings)
    mape_sum = np.sum(np.abs((ratings - mean_rating) / mean_rating))
    mape = mape_sum / ratings.shape[0]
    return mape

%time mape_c(df_fav_100)

Wall time: 5.87 ms


0.11138533165746735

In [160]:
#D
# @numba.njit
def mape_d(df):
    ratings = df.rating.values
    mean_rating = np.mean(ratings)
    mape_sum = sum([abs((rating - mean_rating) / mean_rating) for rating in ratings])

    mape = mape_sum / ratings.shape[0]
    return mape
%time mape_d(df_fav_100)

Wall time: 11.3 ms


0.11138533165746958

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4